<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pydicom
from pydicom.dataset import Dataset, FileMetaDataset
from PIL import Image
import numpy as np
import os

def create_dicom_from_tif(tif_files, output_dir, pixel_spacing=(1.0, 1.0), slice_thickness=1.0):
    for idx, tif_file in enumerate(tif_files):
        # Load TIF image
        image = Image.open(tif_file)
        image_array = np.array(image)

        # Create DICOM dataset
        ds = Dataset()

        # Create and set file meta information
        file_meta = FileMetaDataset()
        file_meta.MediaStorageSOPClassUID = pydicom.uid.SecondaryCaptureImageStorage
        file_meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
        file_meta.ImplementationClassUID = pydicom.uid.generate_uid()
        ds.file_meta = file_meta

        # Set DICOM standard fields for 3D Slicer compatibility
        ds.Modality = 'OT'  # 'OT' means "Other", modify if you have specific modality
        ds.PatientName = "Test^Patient"
        ds.PatientID = "123456"
        ds.StudyInstanceUID = pydicom.uid.generate_uid()
        ds.SeriesInstanceUID = pydicom.uid.generate_uid()
        ds.SOPInstanceUID = pydicom.uid.generate_uid()
        ds.SOPClassUID = pydicom.uid.SecondaryCaptureImageStorage

        # Series and study details
        ds.SeriesNumber = 1
        ds.InstanceNumber = idx + 1
        ds.StudyID = '1'
        ds.FrameOfReferenceUID = pydicom.uid.generate_uid()

        # Set image properties
        ds.SamplesPerPixel = 1  # Grayscale
        ds.PhotometricInterpretation = "MONOCHROME2"
        ds.Rows, ds.Columns = image_array.shape
        ds.PixelSpacing = [str(pixel_spacing[0]), str(pixel_spacing[1])]
        ds.SliceThickness = str(slice_thickness)
        ds.ImagePositionPatient = [0, 0, idx * slice_thickness]  # Z-axis changes by slice_thickness
        ds.ImageOrientationPatient = [1, 0, 0, 0, 1, 0]  # Identity matrix (axial orientation)
        ds.PixelRepresentation = 0  # unsigned integer
        ds.BitsAllocated = 16
        ds.BitsStored = 16
        ds.HighBit = 15
        ds.PixelData = image_array.tobytes()

        # Set additional necessary metadata
        ds.PatientOrientation = ''
        ds.StudyDate = '20240101'  # Example date
        ds.StudyTime = '120000'    # Example time

        # Save as DICOM file
        output_file = os.path.join(output_dir, f"image_{idx+1}.dcm")
        ds.save_as(output_file)
        print(f"Saved {output_file}")

# Example usage
tif_files = ["cell_slice1.tif", "cell_slice2.tif", "cell_slice3.tif"]
output_dir = "./dicom_output"
os.makedirs(output_dir, exist_ok=True)

# Specify pixel spacing and slice thickness (in mm) for 3D Slicer
pixel_spacing = (0.5, 0.5)  # Example: 0.5mm x 0.5mm pixel resolution
slice_thickness = 1.0  # Example: 1mm slice thickness

create_dicom_from_tif(tif_files, output_dir, pixel_spacing, slice_thickness)